In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ECC-Assignment-Q2').getOrCreate()

df = spark.read.csv('/Users/aditisonawane/Desktop/ECC/a2/shot_logs.csv', header=True, inferSchema=True)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 11:02:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df.head(5)

[Row(GAME_ID=21400899, MATCHUP='MAR 04, 2015 - CHA @ BKN', LOCATION='A', W='W', FINAL_MARGIN=24, SHOT_NUMBER=1, PERIOD=1, GAME_CLOCK=datetime.datetime(2023, 4, 13, 1, 9), SHOT_CLOCK=10.8, DRIBBLES=2, TOUCH_TIME=1.9, SHOT_DIST=7.7, PTS_TYPE=2, SHOT_RESULT='made', CLOSEST_DEFENDER='Anderson, Alan', CLOSEST_DEFENDER_PLAYER_ID=101187, CLOSE_DEF_DIST=1.3, FGM=1, PTS=2, player_name='brian roberts', player_id=203148),
 Row(GAME_ID=21400899, MATCHUP='MAR 04, 2015 - CHA @ BKN', LOCATION='A', W='W', FINAL_MARGIN=24, SHOT_NUMBER=2, PERIOD=1, GAME_CLOCK=datetime.datetime(2023, 4, 13, 0, 14), SHOT_CLOCK=3.4, DRIBBLES=0, TOUCH_TIME=0.8, SHOT_DIST=28.2, PTS_TYPE=3, SHOT_RESULT='missed', CLOSEST_DEFENDER='Bogdanovic, Bojan', CLOSEST_DEFENDER_PLAYER_ID=202711, CLOSE_DEF_DIST=6.1, FGM=0, PTS=0, player_name='brian roberts', player_id=203148),
 Row(GAME_ID=21400899, MATCHUP='MAR 04, 2015 - CHA @ BKN', LOCATION='A', W='W', FINAL_MARGIN=24, SHOT_NUMBER=3, PERIOD=1, GAME_CLOCK=datetime.datetime(2023, 4, 13, 

In [3]:
for column in df.columns:
    print(column, ":", "Null values: ", df.filter(df[column].isNull()).count())

GAME_ID : Null values:  0
MATCHUP : Null values:  0
LOCATION : Null values:  0
W : Null values:  0
FINAL_MARGIN : Null values:  0
SHOT_NUMBER : Null values:  0
PERIOD : Null values:  0
GAME_CLOCK : Null values:  0
SHOT_CLOCK : Null values:  5567
DRIBBLES : Null values:  0
TOUCH_TIME : Null values:  0
SHOT_DIST : Null values:  0
PTS_TYPE : Null values:  0
SHOT_RESULT : Null values:  0
CLOSEST_DEFENDER : Null values:  0
CLOSEST_DEFENDER_PLAYER_ID : Null values:  0
CLOSE_DEF_DIST : Null values:  0
FGM : Null values:  0
PTS : Null values:  0
player_name : Null values:  0
player_id : Null values:  0


In [2]:
df = df.na.drop(subset=["SHOT_CLOCK"])

In [13]:
from pyspark.sql.functions import col, when, sum, count, rank
from pyspark.sql.window import Window

df2 = df.groupBy('player_name', 'CLOSEST_DEFENDER')
hit = df2.agg(sum(when(col('SHOT_RESULT') == 'missed', 1).otherwise(0)).alias('Missed Shots'), 
              count('SHOT_RESULT').alias('Total shots missed'))
fear_scoredf = hit.withColumn('Fear Score', when(col('Total shots missed') == 0, 0)
                              .otherwise(col('Missed Shots')/col('Total shots missed')))
fear_scoredf.show()

w = Window.partitionBy('player_name').orderBy(col('Missed Shots').desc())
most_unwanted_defender = fear_scoredf.withColumn('rank', rank().over(w)).filter(col('rank') == 1)
                                     .select('player_name', 'CLOSEST_DEFENDER')

most_unwanted_defender.show()


+----------------+------------------+------------+------------------+------------------+
|     player_name|  CLOSEST_DEFENDER|Missed Shots|Total shots missed|        Fear Score|
+----------------+------------------+------------+------------------+------------------+
|    al jefferson| Hardaway Jr., Tim|           1|                 1|               1.0|
|     cody zeller|     Price, Ronnie|           1|                 1|               1.0|
|       gary neal|     Beal, Bradley|           0|                 3|               0.0|
|       gary neal|     Smart, Marcus|           4|                 4|               1.0|
|gerald henderson|    Bazemore, Kent|           2|                 2|               1.0|
|    kemba walker|     Williams, Lou|           1|                 2|               0.5|
|lance stephenson|    Fournier, Evan|           2|                 2|               1.0|
|  gordon hayward|Aldridge, LaMarcus|           2|                 4|               0.5|
|  gordon hayward|   

In [3]:
#2.2
from pyspark.sql.functions import regexp_replace, avg, max
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler


data = df.select('player_name', 'SHOT_DIST', 'CLOSE_DEF_DIST', 'SHOT_CLOCK', 'SHOT_RESULT')

data = data.withColumn('SHOT_RESULT', regexp_replace('SHOT_RESULT', 'made', '1'))
            .withColumn('SHOT_RESULT', regexp_replace('SHOT_RESULT', 'missed', '0'))
            .withColumn('SHOT_RESULT', col('SHOT_RESULT').cast('float'))

data = data.withColumn('SHOT_DIST', col('SHOT_DIST').cast('float')).withColumn('CLOSE_DEF_DIST',
                    col('CLOSE_DEF_DIST').cast('float')).withColumn('SHOT_CLOCK', col('SHOT_CLOCK').cast('float'))

columns = ['SHOT_DIST', 'CLOSE_DEF_DIST', 'SHOT_CLOCK']

va = VectorAssembler(inputCols=columns, outputCol='features')
nba = va.transform(data).select('player_name', 'features','SHOT_RESULT')

kmeans = KMeans(k=4,seed=1,featuresCol='features')
knba = kmeans.fit(nba)

players = ['james harden', 'chris paul', 'stephen curry', 'lebron james']
nba = nba.filter(col('player_name').isin(players))

pred = knba.transform(nba).select('player_name', 'prediction', 'SHOT_RESULT')
res = pred.groupBy('player_name', 'prediction').agg(avg('SHOT_RESULT').alias('avg_shot_result'))
                                                .orderBy('player_name', 'prediction')

res.show()

bestZone = res.join(res.groupBy('player_name').agg(max('avg_shot_result').alias('max_avg_shot_result')), 
                    on='player_name').filter(col('avg_shot_result') == col('max_avg_shot_result'))
                    .select('player_name', 'prediction', 'avg_shot_result').orderBy('player_name')

bestZone.show()


+-------------+----------+-------------------+
|  player_name|prediction|    avg_shot_result|
+-------------+----------+-------------------+
|   chris paul|         0| 0.4914772727272727|
|   chris paul|         1|0.49504950495049505|
|   chris paul|         2|          0.4296875|
|   chris paul|         3| 0.5563380281690141|
| james harden|         0|0.32978723404255317|
| james harden|         1| 0.5604395604395604|
| james harden|         2|0.43617021276595747|
| james harden|         3|0.48520710059171596|
| lebron james|         0| 0.4124293785310734|
| lebron james|         1| 0.6613545816733067|
| lebron james|         2|0.35789473684210527|
| lebron james|         3| 0.5427350427350427|
|stephen curry|         0| 0.4319654427645788|
|stephen curry|         1| 0.6350710900473934|
|stephen curry|         2| 0.4189189189189189|
|stephen curry|         3| 0.5714285714285714|
+-------------+----------+-------------------+

+-------------+----------+------------------+
|  player_nam